In [1]:
import pandas as pd
import plotly.express as px
import MetaTrader5 as mt5
from datetime import datetime

from utils.mt5_utils import get_ohlc, get_ticks

In [2]:
mt5.initialize()


True

In [19]:
bars = get_ohlc('DE40',  mt5.TIMEFRAME_D1, datetime(2020, 1, 1), datetime.now())

cols = ['time', 'open', 'high', 'low', 'close']
bars = bars[cols]

bars['change'] = bars['close'] - bars['open']
bars['range'] = bars['high'] - bars['low']
bars['day'] = bars['time'].dt.dayofweek
bars['week'] = bars['time'].dt.isocalendar().week

bars

,time,open,high,low,close,change,range,day,week
0,2020-01-02,13199.3,13450.5,13178.5,13445.3,246.0,272.0,3,1
1,2020-01-03,13448.2,13452.9,13118.1,13177.0,-271.2,334.8,4,1
2,2020-01-06,13094.5,13166.7,12947.4,13139.4,44.9,219.3,0,2
3,2020-01-07,13139.5,13283.8,13133.1,13180.5,41.0,150.7,1,2
4,2020-01-08,13175.3,13402.3,12967.0,13381.3,206.0,435.3,2,2
...,...,...,...,...,...,...,...,...,...
884,2023-06-19,16342.9,16351.1,16186.3,16211.2,-131.7,164.8,0,25
885,2023-06-20,16219.7,16219.7,16066.3,16125.7,-94.0,153.4,1,25
886,2023-06-21,16131.1,16156.9,15993.8,15995.9,-135.2,163.1,2,25
887,2023-06-22,16002.7,16008.1,15810.3,15992.3,-10.4,197.8,3,25


In [20]:
px.line(bars, x='time', y='range')